In [ ]:
from tabulate import tabulate
import json
import numpy as np
import h5py
import os
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm

In [ ]:
json_data = '/home/cds-s/workspace/hierarchical_localization/table.json'
path_to_hdf5_datasets = '/media/cds-s/data/Datasets/Habitat/1LXtFkjw3qL_point0/hdf5/'
output_path = '/media/cds-s/data/Datasets/Habitat/table.txt'

In [ ]:
with open(json_data) as f:
    table = json.load(f)

In [ ]:
def quaternion_to_rotation_matrix(qvec):
    r = R.from_quat([qvec[1], qvec[2], qvec[3], qvec[0]])
    result = r.as_matrix()
    result[:3,2] = -result[:3,2]
    result[:3,1] = -result[:3,1]
    return result

In [5]:
angle_thr = 10
dist_thr = 2
table_metrics = {}
for key in tqdm(table.keys()):
    if not(key in table_metrics.keys()):
        table_metrics[key] = {
            'tp'  : 0,
            'all' : 0
        }
    for query in table[key]['queries-dbs']:
        hdf5_filename = '_'.join(query.split('_')[:2]) + '.hdf5'
        num_image = int(query.split('_')[-1])
        hdf5_file = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename), 'r')
        pose_query_gt = np.eye(4)
        is_database = ""
        if query.find('database') != -1:
            is_database = "_base"
        pose_query_gt[:3,:3] = quaternion_to_rotation_matrix(hdf5_file['quat'+is_database][num_image])
        pose_query_gt[:3,3] = hdf5_file['gps'+is_database][num_image]
        pose_query_estimated = table[key]['queries-dbs'][query]['estimated_pose']
        error_pose = np.linalg.inv(pose_query_estimated) @ pose_query_gt
        dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
        r = R.from_matrix(error_pose[:3, :3])
        rotvec = r.as_rotvec()
        angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
        angle_error = abs(180 - abs(angle_error-180))
        if angle_error < angle_thr and dist_error < dist_thr:
            table_metrics[key]['tp'] += 1
        table_metrics[key]['all'] += 1

100%|██████████| 176/176 [01:42<00:00,  1.71it/s]


In [7]:
table = table_metrics

In [9]:
distances = []
angles = []
for key in table.keys():
    distance, _, _, angle, _ = key.split(' ')
    distances.append(float(distance))
    angles.append(float(angle))
distances = sorted(distances)
angles = sorted(list(set(angles)))[::-1]

distances = sorted(list(set(distances)))

table_for_printing = []

set_of_existing_angles = set()

num_angle = 0
num_distance = 0
stroka = []

output_file = open(output_path, 'w')
while num_angle != len(angles):
    for key in table.keys():
        distance, _, _, angle, _ = key.split(' ')
        distance = float(distance)
        angle = float(angle)
        if angle == angles[num_angle] and distance == distances[num_distance]:
            if num_distance == 0:
                if num_angle != len(angles)-1:
                    stroka = [str(int(angles[num_angle+1])) + "°-\n"+ str(int(angles[num_angle]))+"°"]
                else:
                    stroka = ["0°-\n"+ str(int(angles[num_angle])) + " °"]
                stroka.append(str(table[key]["tp"])+"/"+str(table[key]["all"]))
                num_distance += 1
            else:
                num_distance += 1
                stroka.append(str(table[key]["tp"])+"/"+str(table[key]["all"]))
            if num_distance == len(distances):
                stroka_for_txt_file = stroka[1:]
                stroka_for_txt_file = ' '.join(stroka_for_txt_file)
                output_file.write(stroka_for_txt_file + "\n")
                table_for_printing.append(stroka)
                num_distance = 0
                num_angle += 1
                if num_angle == len(angles):
                    break

# print(stroka)

table_for_printing.append([" "] + ['0-\n'+str(distances[0])+" m"] + [str(distances[i])+"-\n"+str(distance)+" m" for i, distance in enumerate(distances[1:])])

print(tabulate(table_for_printing, tablefmt='fancy_grid'))
output_file.close()

╒═══════╤═════════╤═════════╤═════════╤═════════╤═════════╤════════╤═════════╤════════╤════════╤════════╤════════╕
│ 150°- │ 0/0     │ 0/0     │ 0/0     │ 0/0     │ 0/0     │ 0/0    │ 0/0     │ 0/0    │ 0/0    │ 0/0    │ 0/0    │
│ 180°  │         │         │         │         │         │        │         │        │        │        │        │
├───────┼─────────┼─────────┼─────────┼─────────┼─────────┼────────┼─────────┼────────┼────────┼────────┼────────┤
│ 120°- │ 0/929   │ 0/1054  │ 0/544   │ 2/925   │ 9/1001  │ 2/1071 │ 0/1088  │ 0/1088 │ 0/1088 │ 0/279  │ 0/0    │
│ 150°  │         │         │         │         │         │        │         │        │        │        │        │
├───────┼─────────┼─────────┼─────────┼─────────┼─────────┼────────┼─────────┼────────┼────────┼────────┼────────┤
│ 90°-  │ 2/933   │ 13/1057 │ 12/550  │ 15/929  │ 13/1004 │ 9/1069 │ 3/1088  │ 0/1088 │ 0/1088 │ 0/281  │ 0/0    │
│ 120°  │         │         │         │         │         │        │         │  

In [15]:
distances = []
angles = []
for key in table.keys():
    distance, _, _, angle, _ = key.split(' ')
    distances.append(float(distance))
    angles.append(float(angle))
distances = sorted(distances)
angles = sorted(list(set(angles)))[::-1]

distances = sorted(list(set(distances)))

table_for_printing = []

set_of_existing_angles = set()

num_angle = 0
num_distance = 0
stroka = []

output_file = open(output_path, 'w')
while num_angle != len(angles):
    for key in table.keys():
        distance, _, _, angle, _ = key.split(' ')
        distance = float(distance)
        angle = float(angle)
        if angle == angles[num_angle] and distance == distances[num_distance]:
            num_distance += 1
            if table[key]["all"] != 0:
                stroka.append(str(table[key]["tp"]/table[key]["all"]))
            else:
                stroka.append(str(0))
            if num_distance == len(distances):
                stroka_for_txt_file = ' '.join(stroka)
                stroka = []
                output_file.write(stroka_for_txt_file + "\n")
                num_distance = 0
                num_angle += 1
                if num_angle == len(angles):
                    break

output_file.close()

In [16]:
stroka

[]